# Crystal Electric Field (CF) Analysis of the Pr4+ ion
Here the goal is to use collected inelastic neutron scattering (INS) data to determine the ground state crystal field Hamiltonian. The Stevens' coefficients first guess comes from the grid search. I use PyCrystalField in conjunction with LMFIT to find the true values of the Stevens' Coefficients by fitting predicted energy levels to measured one. I then calculate the CF Hamiltonian and predict the compounds thermodynamic properties. Magnetization (M vs H) and Susceptibility (M vs T) data. 

It's important to note that I don't actually use Pr4+ as my central ion, I use Ce3+. The correct orbital values for the Ln4+ oxidized states have not been calculated and tabulated. Since the operators depend on electronic orbital states, I can use an electrically equivalent ion, Ce3+, instead of the actual Pr4+ ion.

In [1]:
import sys
sys.path.append('..')
from JensenTools import *
import seaborn as sns
sns.set()

 *******************************************************
 *                PyCrystalField 2.3.1                 *
 *  Please cite  J. Appl. Cryst. (2021). 54, 356-362   * 
 *    <https://doi.org/10.1107/S160057672001554X>      *
 *******************************************************



##  Energy, magnetization, and susceptibiltiy functions to be made into LMFIT models.

In [2]:
#####################################################################################################################################################################
# Function to be made into an LMFIT model.
def energyFit(B40,B60, B44, B64, B20, numlevels, LS, **kwargs ):
    numlevels = numlevels
    Stev = {} #Creating the Stevens' Coefficients dictionary and assigning values
    Stev['B20'] = B20
    Stev['B40'] = B40
    Stev['B60'] = B60
    Stev['B44'] = B44
    Stev['B64'] = B64

    if kwargs['LS_on']:
        Pr = cef.LS_CFLevels.Bdict(Bdict=Stev, L=3, S=0.5, SpinOrbitCoupling = LS) #Create CF_Levels obejct wtih the given coefficients.
        Pr.diagonalize()
        if kwargs['Kmeans']:
            e = kmeansSort(Pr.eigenvalues,numlevels)[:numlevels-1] #Excluding the highest mode which we did not detect in our INS runs
#             e.append(e[2]/e[1]) #The aforementioned ratio
#             e.append(e[2]/e[0])
        else: 
            e = Pr.eigenvalues
    else:
        Pr = cef.CFLevels.Bdict(Bdict = Stev, ion = kwargs['ion'])
        Pr.diagonalize()
        if kwargs['Kmeans']:   	
            e = kmeansSort(Pr.eigenvalues,numlevels)[:numlevels-1] #Excluding the highest mode which we did not detect in our INS runs
            e.append(e[1]/e[0]) #The aforementioned ratio
        else:
            e =  Pr.eigenvalues
    return e

# Function to be made into an LMFIT model.
def magFit(Pr,a,b,c, HTes, **kwargs ):
    mag = []
    if kwargs['LS_on']:
        for i in HTes:
            mag.append(a*Pr.magnetization(Temp = Temp, Field = [i, 0, 0])[0] + b*Pr.magnetization(Temp = Temp, Field = [0, i, 0])[1] + c*Pr.magnetization(Temp = Temp, Field = [0, 0, i])[2])
    else:
        for i in HTes:
            mag.append(a*Pr.magnetization(Temp = Temp, Field = [i, 0, 0], ion = ion)[0] + b*Pr.magnetization(Temp = Temp, Field = [0, i, 0], ion = ion)[1] + c*Pr.magnetization(Temp = Temp, Field = [0, 0, i], ion = ion)[2])
    return -1*np.array(mag)


# Function to be made into an LMFIT model.
def inverseSusFit(Pr,Ha,T,a,c, **kwargs ):
    mag = []
    if kwargs['LS_on']:
        for i in T:
            mag.append(a*Pr.magnetization(Temp = i, Field = [Ha, 0, 0])[0] + a*Pr.magnetization(Temp = i, Field = [0, Ha, 0])[1] + c*Pr.magnetization(Temp = i, Field = [0, 0, Ha])[2])
    else:
        for i in T:
            mag.append(a*Pr.magnetization(Temp = i, Field = [Ha, 0, 0], ion = ion)[0] + a*Pr.magnetization(Temp = i, Field = [0, Ha, 0], ion = ion)[1] + c*Pr.magnetization(Temp = i, Field = [0, 0, Ha], ion = ion)[2])
    
    return -Ha/np.array(mag)

# Simple function for pritting my parameters after fitting so that I can copy paste the values from output for further iterations.
def paramPrint(fittedparams):
	print()
	for i in fittedparams:
		# print(i, ' = ', i.value)
		print(i, ' = ',fittedparams[i].value )
#####################################################################################################################################################################

## Define kwargs and other important things

In [3]:
# Define important things
#####################################################################################################################################################################
comp = 'Sr2PrO4'
ion = 'Ce3+'
who = 'Arun'
LS_on = True
Kmeans = True
molweight = molweight[comp]
LSValue = 107.5

if LS_on:
	numlevels = 4
	Emeas = [168, 335,385] # The measured INS magnetic modes
else:
	numlevels = 3
	Emeas = [168, 335, 335/168] # The measured INS magnetic modes, only first 2 for J basis
    
# The L,S values are as follows for the Pr4+ ion
L = 3
S = 0.5
#####################################################################################################################################################################

## Load the project's data directories from my DropBox

In [4]:
saveDir = getSaveDir('m',comp = comp)
MHDir = getSaveDir('m',comp = comp, dataType = 'MH')
MTDir = getSaveDir('m',comp = comp, dataType = 'MT')

## Starting values from grid search for both LS and J basis

In [5]:
# From GridSearch For LS
#####################################################################################################################################################################
if LS_on:
	LS = LSValue
	x =  0.03629536921151444
	bpf = -0.6570713391739674
	# Assigning the coefficients from grid search
	# Enforcing cubic constraints as a start
	# and including the B20 term which is needed for tetragonal symmetry
	B40 = bpf
	B60 = x*bpf
	B44 = 5*B40
	B64 = -21*B60
	B20 = 0
#####################################################################################################################################################################

# From GridSearch For J
#####################################################################################################################################################################
if not LS_on:
	x = -1.0000
	bpf = -0.4673
	# Assigning the coefficients from grid search
	# Enforcing cubic constraints as a start
	# and including the B20 term which is needed for tetragonal symmetry	
	B40 = bpf
	B60 = x*bpf
	B44 = 5*B40
	B64 = -21*B60
	B20 = 0
#####################################################################################################################################################################

## Best fit parameters from previous iterations

In [6]:
# #Best Fit LS
# #####################################################################################################################################################################
# if LS_on:
# 	B40  =  -0.6568663783690575
# 	B60  =  -0.02328250024945387
# 	LS   =  107.5
# 	B44  =  -3.1415463304732714
# 	B64  =  0.504906552605772
# 	B20  =  0.4858075931009187
# #####################################################################################################################################################################

# #Fix B20 to different values, check g tensor 

# # Best Fit J
# #####################################################################################################################################################################
# if not LS_on:
# 	# Red Chi = ~5
# 	# B40  =  -0.5572886105373519
# 	# B60  =  0.4673
# 	# B44  =  -3.0342208316734602
# 	# B64  =  -9.8133
# 	# B20  =  12.606195910392971

# 	# # Red Chi = ~.01
# 	B40  =  -0.5572886105373519
# 	B60  =  0.4673
# 	B44  =  -3.0946858584804335
# 	B64  =  -9.8133
# 	B20  =  12.606195720794622
# #####################################################################################################################################################################


## LMFIT model creation, fitting, and creating Stevens' Coefficients dictionary from best fit values.

In [7]:
# Make LMFIT model and fit
# Create stevens coefficients dictionary from fitted parameters
#####################################################################################################################################################################
eModel = Model(energyFit, independent_vars = ['numlevels'])
params = eModel.make_params()

# 
# B40  =  -0.6570713391739674
# B60  =  -0.01909453568222796
# B44  =  -3.6860037014435165
# B64  =  0.5701984705510157
# B20  =  0.4522847388230056
# LS  =  107.5

# Since we only have 4 training points, only 4 parameters can vary at once.
params['B20'].set(value = B20, vary = False)
params['B40'].set(value=B40, vary = False)
params['B60'].set(value=B60, vary = False)
params['B44'].set(value = B44, vary = False)
params['B64'].set(value = B64, vary = False)

if LS_on:
    params['LS'].set(value=LS, vary=False)
    
# Fit model to data
fitted = eModel.fit(Emeas,params, numlevels = numlevels, LS_on = LS_on, Kmeans = Kmeans, ion = ion)
# Create a dictionary of the fitted parameters (stevens coefficients)
stev = {'B40': fitted.params['B40'].value, 'B60': fitted.params['B60'].value, 'B44' : fitted.params['B44'].value, 'B64' : fitted.params['B64'].value, 'B20' :fitted.params['B20'].value }
#####################################################################################################################################################################

# Print the parameters and reduced chi sqr value
#####################################################################################################################################################################
print('\n\nFitted parameters:')
fitted.params.pretty_print()
print('\nReduced Chi Sqr = {}'.format(fitted.result.redchi))
#Uncomment to print out in easy copy paste format
paramPrint(fitted.params)
#####################################################################################################################################################################



Fitted parameters:
Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
B20         0     -inf      inf     None    False     None     None
B40   -0.6571     -inf      inf     None    False     None     None
B44    -3.285     -inf      inf     None    False     None     None
B60  -0.02385     -inf      inf     None    False     None     None
B64    0.5008     -inf      inf     None    False     None     None
LS      107.5     -inf      inf     None    False     None     None

Reduced Chi Sqr = 231.1283309184039

B40  =  -0.6570713391739674
B60  =  -0.023848646853623378
B44  =  -3.285356695869837
B64  =  0.5008215839260909
B20  =  0
LS  =  107.5


## PCF G Tensor

In [8]:
# CF Analysis
#####################################################################################################################################################################
# Create the CFLevels object and diagonalize it
if LS_on:
    Pr = cef.LS_CFLevels.Bdict(Bdict = stev, L = L, S = S, SpinOrbitCoupling=fitted.params['LS'].value)
    Pr.diagonalize()
else:
    Pr = cef.CFLevels.Bdict(Bdict = stev, ion = 'Ce3+')
    Pr.diagonalize()

# Print final matrix
print('\n\nEnergy values as measured by INS (meV): {}'.format(Emeas[:]))
Pr.printEigenvectors()

# Calculate and neatly print G-Tensor using Pandas
gt = Pr.gtensor()
rows = ['gx','gy','gz']
df = pd.DataFrame(gt, columns = rows, index = rows)
print(df)
#####################################################################################################################################################################



Energy values as measured by INS (meV): [168, 335, 385]

 Eigenvalues 	 Eigenvectors
		---------------------------------------------------------------------------------------------------------
0.00000 	|  [ 0.     0.859 -0.302  0.     0.     0.     0.     0.     0.     0.283
 -0.302  0.     0.     0.   ]  |
0.00000 	|  [ 0.     0.     0.    -0.302  0.283  0.     0.     0.     0.     0.
  0.    -0.302  0.859  0.   ]  |
0.00000 	|  [ 0.068  0.     0.     0.     0.     0.     0.    -0.696  0.715  0.
  0.     0.     0.     0.   ]  |
0.00000 	|  [ 0.     0.     0.     0.     0.    -0.715  0.696  0.     0.     0.
  0.     0.     0.    -0.068]  |
171.87001 	|  [ 0.     0.     0.     0.55  -0.631  0.     0.     0.     0.     0.
  0.     0.247  0.488  0.   ]  |
171.87001 	|  [ 0.    -0.488 -0.247  0.     0.     0.     0.     0.     0.     0.631
 -0.55   0.     0.     0.   ]  |
352.68571 	|  [0.645 0.    0.    0.    0.    0.    0.    0.577 0.5   0.    0.    0.
 0.    0.   ]  |
352.68571 	|  [ 

## Loading susceptibility (M vs T) data.

In [9]:
runs = []
for i in os.listdir(MTDir):
    if i.endswith('.DAT') or i.endswith('.dat'): #This was a safeguard against a situation arising at an earlier implementation of my code.
        runs.append(i)

MTdata = {}
# plt.figure()
for i in runs:
    M,H,T,Err,mass,measType = getData(i,MTDir, who = who, dataType = 'MT')
    M = normalize(M,mass,molweight, 'spin')
    Err = normalize(Err,mass,molweight, 'spin')    
    MTdata[measType] = [M,H,T,Err,samplemass]
    # plt.errorbar(H,M, yerr = Err, label = name)
    
runs = []
for i in os.listdir(MHDir):
    if i.endswith('.DAT') or i.endswith('.dat'): #This was a safeguard against a situation arising at an earlier implementation of my code.
        runs.append(i)
        
# Normalizes and stores data as well as plotting in Emu/Oe for all temperatures.
# plt.figure()
MHdata = {}
for i in runs: 
    M, H, Err, mass, T = getData(i,MHDir,who = who, dataType = 'MH')
    M = normalize(M,mass,molweight,'spin')
    Err = normalize(Err,mass,molweight,'spin')
    MHdata[T] = [M,H,Err,mass,i]
#     plt.errorbar(H, M, yerr = Err, label = T)

NameError: name 'samplemass' is not defined

## Calculating PCF susceptibility and plotting against measurement.

In [ ]:
M,H,T,Err,samplemass = MTdata['FC']

MBohr = emuToBohr2(M)
HTes = oeToTesla(H)
X = MBohr/HTes
Xi  = 1/X

if LS_on:
    XPCF = -1*Pr.susceptibility(Temps = T, Field = .1, deltaField = .001)
else:
    XPCF = -1*Pr.susceptibility(Temps = T, Field = .1, deltaField = .001, ion = ion)
    
XiPCF = 1/XPCF

plt.figure()
plt.plot(T,Xi, label = 'Measured H/M')
plt.plot(T,XiPCF, label = 'PCF Susceptibility Powder Average')
plt.xlabel('Temperature (K)')
plt.ylabel('X^-1 (uB^-1 Tesla spin)')
plt.legend()
plt.title('Inverse Susceptbility {} Tesla '.format(.1))
plt.show()

## Loading magnetization data for 20K run. Calculating PCF magnetization (powder average) and plotting the 2.

In [ ]:
# Choosing 20K run
Tmh = '20K'
Temp = getTemp(MHdata[Tmh][-1], who = who)
M, H, Err, mass, filename = MHdata[Tmh]
M = emuToBohr2(M)
H = oeToTesla(H)

mag = []
for i in H:
    if LS_on:
        mag.append(1/3*Pr.magnetization(Temp = Temp, Field = [i, 0, 0])[0] + 1/3*Pr.magnetization(Temp = Temp, Field = [0, i, 0])[1] + 1/3*Pr.magnetization(Temp = Temp, Field = [0, 0, i])[2])
    else:
        mag.append(1/3*Pr.magnetization(Temp = Temp, Field = [i, 0, 0], ion=ion)[0] + 1/3*Pr.magnetization(Temp = Temp, Field = [0, i, 0], ion = ion)[1] + 1/3*Pr.magnetization(Temp = Temp, Field = [0, 0, i], ion = ion)[2])

mag = -1*np.array(mag)

plt.figure()
plt.plot(H,M, label = 'Measured')
plt.plot(H, mag , label = 'PCF Powder Average')
plt.xlabel('Field (T)')
plt.ylabel('M (uB spin^-1)')
plt.legend()
plt.title('Magnetization at T  = {} K'.format(Tmh))
plt.show()

## Fitting inverse susceptibility. The powder averaging coefficients are the parameters.

In [ ]:
# # Make LMFIT model and fit
# # Create stevens coefficients dictionary from fitted parameters
# #####################################################################################################################################################################
# susIModel = Model(inverseSusFit, independent_vars = ['Ha', 'Pr', 'T'])
# params = susIModel.make_params()

# # Bounded
# # params['a'].set(value = 1/3, min = .0001, max = 1)
# # params['b'].set(value = 1/3, min = .0001, max = 1)
# # params['c'].set(value = 1/3, min = .0001, max = 1)
# params['a'].set(value = 1/3, min = .0001, max = 1)
# params.add('c', expr = '1-a' )

# ## Unbounded
# # params['a'].set(value = 1/3)
# # params['b'].set(value = 1/3)
# # params['c'].set(value = 1/3)

# # Fit model to data
# Ha = .1
# XFitted = susIModel.fit(Xi,params,Pr = Pr, Ha = Ha, T = T, LS_on = LS_on, ion = ion)
# XFitted.params.pretty_print()

## Calculate powder averaged magnetization from fitted susceptibiltiy coefficients.

In [ ]:
# # Choosing 20K run
# Tmh = '20K'
# Temp = getTemp(MHdata[Tmh][-1], who = who)
# M, H, Err, mass, filename = MHdata[Tmh]
# M = emuToBohr2(M)
# H = oeToTesla(H)

# mag = []
# for i in H:
#     if LS_on:
#         mag.append(XFitted.params['a']*Pr.magnetization(Temp = Temp, Field = [i, 0, 0])[0] + XFitted.params['a']*Pr.magnetization(Temp = Temp, Field = [0, i, 0])[1] + XFitted.params['c']*Pr.magnetization(Temp = Temp, Field = [0, 0, i])[2])
#     else:
#         mag.append(XFitted.params['a']*Pr.magnetization(Temp = Temp, Field = [i, 0, 0], ion=ion)[0] + XFitted.params['a']*Pr.magnetization(Temp = Temp, Field = [0, i, 0], ion = ion)[1] + XFitted.params['c']*Pr.magnetization(Temp = Temp, Field = [0, 0, i], ion = ion)[2])

# mag = -1*np.array(mag)


# plt.figure()
# plt.plot(T,Xi, label = 'Measured H/M')
# plt.plot(T,XFitted.best_fit, label = 'PCF Powder Average Fit')
# plt.xlabel('Temperature (K)')
# plt.ylabel('X^-1 (uB^-1 Tesla spin)')
# plt.legend()
# plt.title('Inverse Susceptbility {} Tesla '.format(.1))
# plt.show()

# plt.figure()
# plt.plot(H,M, label = 'Measured M')
# plt.plot(H, mag , label = 'PCF Powder Average Fit Extrapolated')
# plt.xlabel('Field (T)')
# plt.ylabel('M (uB spin^-1)')
# plt.legend()
# plt.title('Magnetization at T  = {} K'.format(Tmh))
# plt.show()

## PCF Neutron Spectrum
Define: an energy array that will be used to calculate lineshape; neutron incident energy, Ei (mev); temperature (K); instrument resolution (meV).

Instrument resolution is typically ~3% - 5% of the Ei.

Lineshape fitting would be ideal for determining the CF Hamiltonian; however, nuances arise. If not all magnetic modes are seen at a single Ei then certain scaling has to be taken into account to adjust the intensities of the signal. For this reason we decided to go down the route of fitting to energy levels.

In [ ]:
# ## PCF Neutron Spectrum
#####################################################################################################################################################################
Ei = 700
Temp = 4.82
res = 9
energy = np.linspace(.01,Ei,1000)

CalculatedSpectrum = Pr.neutronSpectrum(energy, Temp=Temp, Ei=Ei, ResFunc = lambda x: res )
# ResFunc = lambda x: 9 if (energy < 200) else 21
plt.figure()
plt.plot(energy,CalculatedSpectrum)
plt.ylabel('Intensity (arb. units)')
plt.xlabel('Energy (meV)')
plt.title('PCF Spectrum: Ei = {}meV, Temp = {}K, Res = {}'.format(Ei,Temp,res))
plt.show()
#####################################################################################################################################################################

## Print Hamiltonian in LaTex Format

## Convert our fitted Stevens' to Wybourne and print

In [ ]:
print()
Pr.printLaTexEigenvectors()
print()

wyb = cef.StevensToWybourne('Ce3+',stev, LS=True)
print("Fitted coefficients in Wybourne's")
print(wyb)